<a href="https://colab.research.google.com/github/sunkusun9/ML2022_1/blob/main/TPS202208.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro

[Kaggle의 Tabular Playground 2022-08월 과제](https://www.kaggle.com/competitions/tabular-playground-series-aug-2022) 를 기반으로 만들어진 문제입니다.

액체가 엎질러졌을 제품에 따라서 여러 조건에 따라, 물기를 완전히 흡수하는데 성공할지 아니면 실패했는지에 대한 테스트하는 실험에 대한 데이터입니다. 


|속성명|설명|타입|
|------|----|----|
|ID|고유번호|int|
|loading|흡수할 용량|float|
|product_code|상품코드|string|
|attribute_0|구성물질 1|string|
|attribute_1|구성물질 2|string|
|attribute_2|구성물질 1 함량|string|
|attribute_3|구성물질 2 함량|string|
|measure_0~2|측정치 0~2|int|
|measure_3~17|측정치 3~17|float|
|failure|실패여부|boolean| 

# Download

Kaggle API를 이용하여 Kaggle에 등록된 ID와 인증키를 가지고 데이터를 다운로드

In [10]:
import os

os.environ['KAGGLE_USERNAME'] = ''
os.environ['KAGGLE_KEY'] = ''

# train.csv가 없으면 Kaggle에서 데이터를 다운로드 받는다.
if not os.path.exists('train.csv'):
    !kaggle competitions download -c tabular-playground-series-aug-2022
    !unzip tabular-playground-series-aug-2022.zip
    !rm tabular-playground-series-aug-2022.zip

  0% 0.00/2.27M [00:00<?, ?B/s]
100% 2.27M/2.27M [00:00<00:00, 164MB/s]
Archive:  tabular-playground-series-aug-2022.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [35]:
!head train.csv

id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
0,A,80.1,material_7,material_8,9,5,7,8,4,18.04,12.518,15.748,19.292,11.739,20.155,10.672,15.859,17.594,15.193,15.029,,13.034,14.684,764.1,0
1,A,84.89,material_7,material_8,9,5,14,3,3,18.213,11.54,17.717,17.893,12.748,17.889,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,A,82.43,material_7,material_8,9,5,12,1,5,18.057,11.652,16.738,18.24,12.718,18.288,12.715,15.607,,13.798,16.711,18.631,14.094,17.946,663.376,0
3,A,101.07,material_7,material_8,9,5,13,2,6,17.295,11.188,18.576,18.339,12.583,19.06,12.471,16.346,18.377,10.02,15.25,15.562,16.154,17.172,826.282,0
4,A,188.06,material_7,material_8,9,5,9,2,8,19.346,12.95,16.99,15.746,11.306,1

# Preparation

In [36]:
import pandas as pd
import numpy as np
import sklearn
import scipy

for i in [pd, np, sklearn, scipy]:
    print(i.__name__, i.__version__)

pandas 1.3.5
numpy 1.21.6
sklearn 1.0.2
scipy 1.7.3


In [225]:
df_org = pd.read_csv('train.csv', dtype={'failure': 'int8', 'attribute_2': 'int8', 'attribute_3': 'int8'}, index_col='id')

In [38]:
df_org.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26570 entries, 0 to 26569
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              26570 non-null  int64  
 1   product_code    26570 non-null  object 
 2   loading         26320 non-null  float64
 3   attribute_0     26570 non-null  object 
 4   attribute_1     26570 non-null  object 
 5   attribute_2     26570 non-null  int8   
 6   attribute_3     26570 non-null  int8   
 7   measurement_0   26570 non-null  int64  
 8   measurement_1   26570 non-null  int64  
 9   measurement_2   26570 non-null  int64  
 10  measurement_3   26189 non-null  float64
 11  measurement_4   26032 non-null  float64
 12  measurement_5   25894 non-null  float64
 13  measurement_6   25774 non-null  float64
 14  measurement_7   25633 non-null  float64
 15  measurement_8   25522 non-null  float64
 16  measurement_9   25343 non-null  float64
 17  measurement_10  25270 non-null 

In [39]:
df_org.describe().T

,count,mean,std,min,25%,50%,75%,max
id,26570.0,13284.500000,7670.242662,0.000,6642.25000,13284.5000,19926.75000,26569.000
loading,26320.0,127.826233,39.030020,33.160,99.98750,122.3900,149.15250,385.860
attribute_2,26570.0,6.754046,1.471852,5.000,6.00000,6.0000,8.00000,9.000
attribute_3,26570.0,7.240459,1.456493,5.000,6.00000,8.0000,8.00000,9.000
measurement_0,26570.0,7.415883,4.116690,0.000,4.00000,7.0000,10.00000,29.000
measurement_1,26570.0,8.232518,4.199401,0.000,5.00000,8.0000,11.00000,29.000
measurement_2,26570.0,6.256568,3.309109,0.000,4.00000,6.0000,8.00000,24.000
measurement_3,26189.0,17.791528,1.001200,13.968,17.11700,17.7870,18.46900,21.499
measurement_4,26032.0,11.731988,0.996085,8.008,11.05100,11.7330,12.41000,16.484
measurement_5,25894.0,17.127804,0.996414,12.073,16.44300,17.1320,17.80500,21.425


In [40]:
df_org[['product_code', 'attribute_0', 'attribute_1', 'attribute_2', 'attribute_3']].value_counts()

product_code  attribute_0  attribute_1  attribute_2  attribute_3
C             material_7   material_8   5            8              5765
E             material_7   material_6   6            9              5343
B             material_5   material_5   8            8              5250
D             material_7   material_5   6            6              5112
A             material_7   material_8   9            5              5100
dtype: int64

In [218]:
df_test = pd.read_csv('test.csv', dtype={'failure': 'int8', 'attribute_2': 'int8', 'attribute_3': 'int8'}, index_col='id')
df_test.head()

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
0,26570,F,119.57,material_5,material_6,6,4,6,9,6,...,18.654,10.802,15.909,18.070,13.772,13.659,16.825,13.742,17.710,634.612
1,26571,F,113.51,material_5,material_6,6,4,11,8,0,...,19.368,12.032,13.998,NaN,12.473,17.468,16.708,14.776,14.102,537.037
2,26572,F,112.16,material_5,material_6,6,4,8,12,4,...,17.774,11.743,17.046,18.086,10.907,13.363,15.737,17.065,16.021,658.995
3,26573,F,112.72,material_5,material_6,6,4,8,11,10,...,18.948,11.790,18.165,16.163,10.933,15.501,15.667,12.620,16.111,594.301
4,26574,F,208.00,material_5,material_6,6,4,14,16,8,...,19.141,12.370,14.578,17.849,11.941,16.070,16.183,13.324,17.150,801.044


In [194]:
df_test.describe().T

,count,mean,std,min,25%,50%,75%,max
id,20775.0,36957.000000,5997.370257,26570.000,31763.50000,36957.0000,42150.5000,47344.000
loading,20552.0,127.634895,39.154642,37.700,99.47000,122.1100,148.8400,385.570
attribute_2,20775.0,7.733959,1.308535,6.000,6.00000,7.0000,9.0000,9.000
attribute_3,20775.0,6.196823,1.917478,4.000,4.00000,5.0000,7.0000,9.000
measurement_0,20775.0,7.453574,4.274477,0.000,4.00000,7.0000,10.0000,30.000
measurement_1,20775.0,8.962407,4.334200,0.000,6.00000,9.0000,12.0000,33.000
measurement_2,20775.0,6.126931,3.835881,0.000,3.00000,6.0000,8.0000,28.000
measurement_3,20446.0,17.793466,1.002206,13.565,17.11900,17.7890,18.4780,21.389
measurement_4,20366.0,11.727331,1.006834,7.384,11.04825,11.7290,12.4110,15.623
measurement_5,20267.0,17.138080,1.008714,12.215,16.45700,17.1320,17.8245,21.681


In [195]:
df_test[['product_code', 'attribute_0', 'attribute_1', 'attribute_2', 'attribute_3']].value_counts()

product_code  attribute_0  attribute_1  attribute_2  attribute_3
F             material_5   material_6   6            4              5422
I             material_7   material_5   9            5              5228
G             material_5   material_6   9            7              5107
H             material_7   material_7   7            9              5018
dtype: int64

In [44]:
df  = df_org.copy()

# 전처리

measurement_17는 failure를 유추하기 위해 product_code 별로 measurement3 ~ measurement_9을 최적의 선형적인 조합으로 기기에서 만들어 주는 값이다. 즉 product_code 별로 measurement3 ~ mesuremenet_9을 입력으로 measurement_17을 선형회귀모델로 학습하면 measurement_17을 복원하는 모델이 된다. 하지만 실험 기기가 종종 오류를 일으켜 measurement_17에 결측이 발생한다. 이 정보를 바탕으로 복원을 위한 선형회귀모델을 만들어 measurement_17의 결측치를 보간하고자 한다.

※ 본 처리 과정은  [Perfect Positive Correlation with measurement_17](https://www.kaggle.com/competitions/tabular-playground-series-aug-2022/discussion/343939) 을 기반으로 만들어 졌습니다.

## 단계 1

measurement_3~9 중에서 결측치가 전혀없는 examine 데이터프레임을 만든다.

In [45]:
m_cols = ['measurement_{}'.format(i) for i in range(3, 10)] # 편의를 위해 measurement3~measurement9까지 컬럼값을 담는 리스트를 만든다.
df_examine = df.loc[df[m_cols + ['measurement_17']].notna().all(axis=1)] # m_cols와 measuement_17에서 결측치가 없는 데이터프레임 df_examine을 만든다.
df['measurement_17'].isna().sum() # measurement_17의 결측치 확인

2284

## 단계 2

examine 데이터프레임을 활용하여, product_code 별로 measurement_3 ~ measurement9  속성을 입력으로 하여 measurement_17을 맞추는 선형회귀모델을 만들고, 이 모델들을 활용하여 복원할 수 있는 measurement_17 결측치를 최대한 복원한다. 그리고 이 데이터프레임을 basetable이라고 한다.

- 선형 모델읜 sklearn.linear_regression.Lasso를 사용하고, 모든 설정은 기본값을 사용한다.

- 회귀계수의 값이 0인 속성은 measurement_17과 전혀 관련이 없다고 간주한다. 

- 절편을 제외한 각 속성에 해당하는 회귀계수의 총합은 1203.35(소수점 2째 자리까지 반올림)이다.

 Guide. 속성명을 열의 이름으로, product_code를 행의 Index로 하여 각각의 값이 회귀계수인 데이터프레임 coef를 만들고 여기서 합을 내도록함을 권장.
 > 단계 3에서도 활용할수 도 있음

* Hint
    - 복원 후 measurement_17의 결측치는 341개로 줄어 든다.

In [46]:
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score

In [47]:
lr = Lasso()
pc_list = [] # coef 데이터프레임 구성을 위해 처리한 product_code 저장 
coef_list = [] # 마찬가지로 회귀계수 저장
impute_cnt = []
for k, v in df_examine.groupby('product_code'): # groupby를 이용하여 product_code 별로 구분하여 데이터프레임의 행들을 나누어 가져온다.
    lr.fit(v[m_cols],v['measurement_17']) # 학습
    print(k, lr.coef_, r2_score(v['measurement_17'], lr.predict(v[m_cols]))) # 결과 확인
    coef_ = lr.coef_.copy() # 복사
    avail_cols = np.array(m_cols)[coef_ != 0] # 연관성이 있는 속성명을 뽑아 낸다, 작업을 편하게 하기 위해 list를 numpy array로 바꾼다.
    pc_list.append(k) # 처리한 product_code를 저장
    coef_list.append(coef_) # 도출한 회귀계수 저장 
    # 현재 복원 스텝을 진행하는 product_code 에서 연관성이 있는 속성이 결측이 없고 measurement_17은 결측인 행을 뽑기 위한  Boolean 인덱싱 행을  만든다.
    target_cond = (df['product_code'] == k) & df[avail_cols].notna().all(axis=1) & df['measurement_17'].isna()
    impute_cnt.append(target_cond.sum())
    # 결측치를 예측하고 입력한다. loc를 행을 선택후 값을 지정할 때에, 대입하는 값의 수가 일치하면, 인덱스와 상관없이 순서대로 대입이 되게 된다. 
    df.loc[target_cond, 'measurement_17'] = lr.predict(df.loc[target_cond][m_cols].fillna(0))
df_coef = pd.DataFrame(np.vstack(coef_list), columns=lr.feature_names_in_, index=pc_list) # coef 데이터프레밍 생성
df_coef.head()

A [-0.         13.88990012 58.79968924 25.78318324 24.15966165 76.39890398
  0.        ] 0.9995502992956551
B [17.63821044 43.43821149 33.96906643  0.         86.45728502  0.
 25.76855271] 0.9995615957156855
C [ 0.          0.         51.42237822  6.15590156 40.67515231 90.18778957
 35.52042918] 0.9996223694466843
D [23.70932099  0.         62.67195241 92.76751898 47.31637707 56.14618422
  0.        ] 0.9997247049648755
E [ 0.         57.69072149 65.64540559 76.06431605  0.         72.15415319
 18.9206499 ] 0.9997300196455193


,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9
A,-0.000000,13.889900,58.799689,25.783183,24.159662,76.398904,0.000000
B,17.638210,43.438211,33.969066,0.000000,86.457285,0.000000,25.768553
C,0.000000,0.000000,51.422378,6.155902,40.675152,90.187790,35.520429
D,23.709321,0.000000,62.671952,92.767519,47.316377,56.146184,0.000000
E,0.000000,57.690721,65.645406,76.064316,0.000000,72.154153,18.920650


In [48]:
df['measurement_17'].isna().sum(), df_coef.sum().sum()

(341, 1203.350915049366)

In [49]:
df_basetable = df.copy()

## 단계 3 (Optional)

※ 본 단계는 구현의 난이도가 상당하여 Optional 단계로 하게 되었고 더 깊은 연습을 하시고자 하는 분들을 위해 만든 것이니, 단계에 영향이 없도록, basetable에서 optional 데이터프레임을 만들고 진행토록합니다. 


위에서 발견한 거의 완벽한 선형적 관계를 이용하여 measurement_17 이외에도 복원할 수 있는 값은 모두 복원하고자 한다. measurement_3~measurement_9 까지 복원하시오.

- sklearn.linear_model.LinearRegression을 사용한다.

- optional 데이터프레임에서 결측을 제거하여 examine 데이터프레임을 새로 만들고 이를 활용하여 선형모델을 만든다.

- 결측 처리를 완료하면, measurement_3 ~ measurement_9 까지의 결측치는 모두 2294건이 된다.

In [50]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [51]:
df_optional =  df_basetable.copy()
df_examine = df_optional.dropna()

In [52]:
df_optional[m_cols].isna().sum().sum()

5603

In [53]:
# product_code 별로 회귀 계수값을 살펴 보면, 위에서 언급했듯이 전혀 연관성이 없느 속성들이 있다. 이들은 다른 값을 이용해도 복원할 수 없다.
# 예를 들면 product_code가 measurement_3, measurement_9 는 복원할 수 없고 다른 값들은 자신을 제외한 다른 속성값과 measurement_17을 이용한다면,
# 입력값에 결측이 없는 경우라면 완전히 복원이 가능하다.
df_coef

,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9
A,-0.000000,13.889900,58.799689,25.783183,24.159662,76.398904,0.000000
B,17.638210,43.438211,33.969066,0.000000,86.457285,0.000000,25.768553
C,0.000000,0.000000,51.422378,6.155902,40.675152,90.187790,35.520429
D,23.709321,0.000000,62.671952,92.767519,47.316377,56.146184,0.000000
E,0.000000,57.690721,65.645406,76.064316,0.000000,72.154153,18.920650


In [54]:
lr = LinearRegression()
for k, v in df_coef.iterrows():
    rel_cols = df_coef.columns[v.values != 0] # 복원을 할 수 있는 열, 즉 회귀계수가 0이 아닌 열 들을 뽑아 보자.
    print(k, v.values, rel_cols)
    df_prod = df_examine.loc[df_examine['product_code'] == k]
    train_error = []
    recover_cnt = []
    for col in rel_cols: # 복원할 수 있는 열 들을 하나하나씩 회귀모델을 만들어 복원작업을 돌리자.
        X_cols = [i for i in rel_cols if col != i] + ['measurement_17'] # 복원 대상을 제외한 입력 속성들만 뽑자, measurement_17은 모두 넣도록 하자.
        lr.fit(df_prod[X_cols], df_prod[col])
        target_cond = df_optional[col].isna() & df_optional[X_cols].notna().all(axis=1) & (df_optional['product_code'] == k)
        if target_cond.sum() == 0:
            continue
        train_error.append(mean_squared_error(df_prod[col], lr.predict(df_prod[X_cols])))
        recover_cnt.append(target_cond.sum())
        df_optional.loc[target_cond, col] = lr.predict(df_optional.loc[target_cond, X_cols].fillna(0))
    print(train_error, recover_cnt)

A [-0.         13.88990012 58.79968924 25.78318324 24.15966165 76.39890398
  0.        ] Index(['measurement_4', 'measurement_5', 'measurement_6', 'measurement_7',
       'measurement_8'],
      dtype='object')
[0.0026022170944758117, 0.0001601881723599166, 0.000803429963261852, 0.0009057186446917824, 9.563857338564942e-05] [83, 120, 148, 140, 154]
B [17.63821044 43.43821149 33.96906643  0.         86.45728502  0.
 25.76855271] Index(['measurement_3', 'measurement_4', 'measurement_5', 'measurement_7',
       'measurement_9'],
      dtype='object')
[0.0017148765648858958, 0.000297973790041269, 0.0004823720320238151, 7.68761650253593e-05, 0.0008253672868783111] [68, 79, 91, 156, 210]
C [ 0.          0.         51.42237822  6.15590156 40.67515231 90.18778957
 35.52042918] Index(['measurement_5', 'measurement_6', 'measurement_7', 'measurement_8',
       'measurement_9'],
      dtype='object')
[0.00021271769731825171, 0.011438749521479828, 0.0003373864961868344, 7.030800228130399e-05, 0.000

In [55]:
df_optional[m_cols].isna().sum().sum()

2294

# 문제 1.

(basetable을 사용) measurement_3~16 까지 결측 여부가 failure에 영향이 있는 지를 파악하고자 한다.

이를 확인하기 위하여, 

n이 3 부터 16까지, 다음의 검증을 수행한다.

$H_0: P(measurement\_\{n\}=Missing|failure=True) = P(measurement\_\{n\}=Missing|failure=False)$

$H_1: P(measurement\_\{n\}=Missing|failure=True) \not= P(measurement\_\{n\}=Missing|failure=False)$

※ 본 문제는 kaggle의 [TPSAUG22 EDA which makes sense](https://www.kaggle.com/code/ambrosm/tpsaug22-eda-which-makes-sense) 코드 분석을 기반으로 만들어졌습니다.


## 단계 1.

우선, measurement_3 속성 하나로 위 검증을 시행하자.

P_missing_true는 failure가 1일 때 measurement_3가 결측일 확률

P_missing_false는 failure가 0 일때 measurement_3가 결측일 확률


이라고 할 때 이 값을 구하라.

In [56]:
df_prob = pd.crosstab(df_basetable['measurement_3'].isna(), df_basetable['failure']) # 비율을 구하기 위해 crosstab으로 빈도를 구한다.
df_prob

failure,0,1
measurement_3,,
False,20601,5588
True,320,61


In [57]:
s_samples = df_prob.sum(axis=0) # 성공여부에 대한 합을 구한다.
prob = (df_prob.loc[True] / s_samples).values # 비율을 구한다
P_missing_false= prob[0]
P_missing_true = prob[1]
s_samples, P_missing_false, P_missing_true

(failure
 0    20921
 1     5649
 dtype: int64, 0.01529563596386406, 0.010798371393166932)

## 단계 2

위 가설은 다음과 같이 

$H_0: P\_missing\_false - P\_missing\_true = 0$

$H_1: P\_missing\_false - P\_missing\_true \not= 0$

으로 바꿀 수 있다. 

중심극한정리(Central Limit Theorem)이 충분히 성립함이 이견이 없다고 할 때, 

P_missing_false - P_missing_true의 분산을 구하고 이를 바탕으로 구하고, 

정규분포를 사용하여 위 가설에 대한 p-value를 구하라.

* Hint: scipy.stats.norm 모듈을 활용

In [58]:
# 위 사건은 베르눌리 분포에 해당하고, P_missing_false_var는 확률이다. 이에 대한 분산은 p(1-p)를 이용
# 샘플 평균에 대한(P_missing_false는 missing을 1로 할 때에 대한 평균이라고 할 수 있다) 분산은 모분산/샘플 수이다.
P_missing_false_var = (P_missing_false) * (1 - P_missing_false) / s_samples[0] 
P_missing_true_var = (P_missing_true) * (1 - P_missing_true) / s_samples[1]

P_var = P_missing_false_var + P_missing_true_var # 두 사건이 독립이므로: VAR(X-Y) = VAR(X) + VAR(-Y) = VAR(X) + VAR(Y)

In [59]:
from scipy.stats import norm

In [60]:
# 양측 검증이므로 절대값을 취하여 작은 값의 누적확률을 구하고 여기에 2를 곱한다.
# norm.cdf의 scale은 표준 편차임을 유의하자.
norm.cdf(-abs(P_missing_true - P_missing_false), loc=0, scale=P_var ** 0.5) * 2

0.005381119615351366

## 단계 3

measuremenet_4 ~ 9 까지 위 검증을 반복하고 결측치가 유의한 차이를 보인 속성의 p_value의 합을 구하여 A라고 한다. (유의 수준은 5%로 한다.)

결측치에 대한 유의한 차이를 보이는 속성은 총 두 개이고, 속성 번호가 작은 것에 대한 결측유무의 속성명을 na_1으로 큰 것을 na_2로 하는 속성 2개을 (basetable)에 추가하고 (결측: 1, 비결측: 0), 이 데이터셋의 명을 prob_1 이라고 한다

In [61]:
sel_na_feat = [] # 유의미한 속성명을 저장한다.
p_val_sum = 0 # 유의미한 속성의 pvalue를 저장한다.
for i in range(3, 10):
    df_prob = pd.crosstab(df_basetable['measurement_{}'.format(i)].isna(), df_basetable['failure']) 
    s_samples = df_prob.sum(axis=0)
    prob = (df_prob.loc[True] / s_samples).values
    P_missing_false= prob[0]
    P_missing_true = prob[1]
    P_missing_false_var = (P_missing_false) * (1 - P_missing_false) / s_samples[0] 
    P_missing_true_var = (P_missing_true) * (1 - P_missing_true) / s_samples[1]

    P_var = P_missing_false_var + P_missing_true_var 

    p_val = norm.cdf(-abs(P_missing_true - P_missing_false), loc=0, scale=P_var ** 0.5) * 2 
    if (p_val < 0.05):
        print(i, p_val)
        sel_na_feat.append(i)
        p_val_sum += p_val
    else:
        print(i, p_val, "x")

3 0.005381119615351366
4 0.1640078890010257 x
5 0.011640088208378058
6 0.8771576941613572 x
7 0.8566216983356773 x
8 0.708683049716051 x
9 0.12439234283366417 x


A의 값을 소수점 3째 짜리까지 반올림하여 출력한다.

Ex) 0.052

In [62]:
round(p_val_sum, 3)

0.017

In [63]:
df_prob_1 = df_basetable.copy()
df_prob_1['na_1'] = df_basetable['measurement_{}'.format(sel_na_feat[0])].isna() * 1
df_prob_1['na_2'] = df_basetable['measurement_{}'.format(sel_na_feat[1])].isna() * 1
df_prob_1[['na_1', 'na_2']].describe()

,na_1,na_2
count,26570.000000,26570.000000
mean,0.014339,0.025442
std,0.118888,0.157467
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,1.000000,1.000000


## 단계 4 (Optional)

measurement_3~measurement_16까지, 결측치수가 failure와 연관성이 있을지 검증하시오.

In [64]:
from scipy.stats import chi2_contingency

In [65]:
df_conti_na = pd.crosstab(index=df_prob_1[['measurement_{}'.format(i) for i in range(3, 17)]].isna().sum(axis=1),
                          columns=df_prob_1['failure'])
chi2_contingency(df_conti_na)

(9.660337889713894,
 0.1397040591113054,
 6,
 array([[1.05707349e+04, 2.85426515e+03],
        [7.35738893e+03, 1.98661107e+03],
        [2.43382804e+03, 6.57171961e+02],
        [5.00781182e+02, 1.35218818e+02],
        [5.19678585e+01, 1.40321415e+01],
        [5.51174257e+00, 1.48825743e+00],
        [7.87391795e-01, 2.12608205e-01]]))

# 문제 2



두가지 사항을  확인하고자 한다.

첫째는 실험에 대해 공평하게 진행이 됐는지를 의문을 가지고 있어 product_code에 대해서 loading을 동일한 조건으로 진행했는지를 조사하라.

분산분석(ANOVA)을 통해 진행을 해보고, 조사의 적합성을 살펴보기 위해, 분산분석의 가정에 얼마나 부합하는지를 보고하라고 덧붙였다. 

둘째는, 조사결과 attribute_0와 attribute_1은 failure와는 관계가 있는지를 조사하는데,해당 재료들의 조합은 이전의 여러 실험에서 성능에 차이가 없다는 결과가 있어 유의 수준을 1%로 정했다.


## 단계 1

prob_1에서 loading 값이 결측이 아닌 값을 뽑아 prob_2 데이터프레임을 만들고, 검증은 prob_2를 통해 진행한다.

prob_2에 loading 속성에 로그 함수를 적용하여 loading_log 컬럼을 만들어, 좌측으로 치우친 loading의 분포를 보다 정규분포 형태로 변환을 한다.

- np.log 함수를 사용한다.

In [66]:
df_prob_2 = df_prob_1.loc[df_prob_1['loading'].notna()].copy()
df_prob_2['loading_log'] = np.log(df_prob_2['loading'])

## 단계 2

(prob_1을 이용) 일원분산분석을 진행을 위해서, loading_log 열이 일원산분산분석의 가정인  product_code 내에서 동일한 분산이 성립하는지 조사하고자 한다.

Bartlett 검정 방법으로, 조사했을 때의  pvalue를 구하고 이 값을 B라고 한다.

- scipy.stats.bartlett 함수를 이용라.

In [67]:
from scipy.stats import bartlett

In [68]:
# 무식
v_A = df_prob_2.loc[df_prob_2['product_code'] == "A", 'loading_log']
v_B = df_prob_2.loc[df_prob_2['product_code'] == "B", 'loading_log']
v_C = df_prob_2.loc[df_prob_2['product_code'] == "C", 'loading_log']
v_D = df_prob_2.loc[df_prob_2['product_code'] == "D", 'loading_log']
v_E = df_prob_2.loc[df_prob_2['product_code'] == "E", 'loading_log']

bartlett(v_A, v_B, v_C, v_D, v_E)

BartlettResult(statistic=3.0374125206089695, pvalue=0.5515841437603531)

In [69]:
# 쪼금 쎄련
v_list = []
for i in df_prob_2['product_code'].unique():
    v_list.append(df_prob_2.loc[df_prob_2['product_code'] == i, "loading_log"])
bartlett(*v_list)

BartlettResult(statistic=3.0374125206089695, pvalue=0.5515841437603531)

In [70]:
# 쎄련
bartlett(*df_prob_2.groupby('product_code')['loading_log'].agg(list).tolist())

BartlettResult(statistic=3.0374125206089695, pvalue=0.5515841437603531)

In [71]:
res = bartlett(*df_prob_2.groupby('product_code')['loading_log'].agg(list).tolist())
B = res.pvalue

## 단계 3

loading_log가 product_code 각각에 대해 정규성을 지니고 있는지 확인하고자 한다. jarque_bera test로 검증하고 결과의 p-value가 0.05가 넘는 열의 수를 C라고 하자.

- scipy.stats.jarque_bera 함수를 이용하라.

In [72]:
from scipy.stats import jarque_bera

In [73]:
# 쪼금 세련 버젼
v_list = []
for i in df_prob_2['product_code'].unique():
    v_list.append(jarque_bera(df_prob_2.loc[df_prob_2['product_code'] == i, "loading_log"])[1])
C = np.sum(np.array(v_list) > 0.05)
C

4

In [74]:
# 쎄련 버젼
C = (df_prob_2.groupby('product_code')['loading_log'].apply(lambda x: jarque_bera(x)[1]) > 0.05).sum()
C

4

## 단계 4

product_code에 대해서 분산분석을 통해서 loading_log 평균의 차이가 나는 지를 확인하고, 해당 검증의 pvalue를 D라고 한다.

- scipy.stats.f_oneway 를 이용하라.

In [75]:
from scipy.stats import f_oneway

In [76]:
res = f_oneway(*df_prob_2.groupby('product_code')['loading_log'].agg(list).tolist())
D = res.pvalue
D

0.6738197224743324

## 단계 5

chi2 분포를 이용한 동질성 검사를 통해 attirubute_0, attribute_1의 조합이 failure에 영향이  있는지 chi2 동질석 검사를 진행하라. 연관성이 있다고 할 수 있으면 E 값은 1로 아니면 0으로 한다.

- 유의 수준은 1%로 한다.

- scipy.stats.chi2_contingency 를 이용하고, correction은 False로 한다.

In [77]:
from scipy.stats import chi2_contingency

In [78]:
df_conti = pd.crosstab(index=df_prob_1['attribute_0'].str.cat(df_prob_1['attribute_1']), columns=df_prob_1['failure'])
res = chi2_contingency(df_conti)
E = 1 if res[0] < 0.01 else 0
res, E

((9.049932287226607,
  0.028634323451681147,
  3,
  array([[4133.8069251 , 1116.1930749 ],
         [4025.14685736, 1086.85314264],
         [4207.03436206, 1135.96563794],
         [8555.01185548, 2309.98814452]])),
 0)

B + C + D + E 의 값을 소수점 3째 자리에서 반올림하여 2째 자리까지 출력하라. 합을 구한 후에 반올림을 진행해야 한다는 점을 유의하시오.

예) 3.16

In [79]:
B + C + D + E

5.225403866234686

# 문제 3

로지스틱회귀모델로 수치형 measurement_0 ~ measurement_17과 loading와 불린형인 na_1, na_2 입력으로 하고, failure를 타깃으로 하는 분류 모델을 제작을 했는데, 제 성능을 내지 못했다.  다른 모델을 사용해보기 전에 효과가 있을 것이라고 생각되는 속성인 loading과 measurement_17으로 시작하여 효과가 있는 조합을 찾고자 한다. 선택되지 않은 속성을 하나 씩만 넣어보고 가장 효과가 큰 속성이 이전 단계보다 좋은 성능 냈다면 해당 속성을 추가 하고 선택 과정을 반복하고 그렇지 않으면 선택을 중단하는 전진선택법을 취하여 속성 조합을 찾기 위해 다음 단계를 진행하라.

대상지표는 AUC(Area Under the ROC Curve) 를 사용한다.

- sklearn.metrics.roc_auc_score

In [80]:
from sklearn.metrics import roc_auc_score

## 단계 1

prob_1 에서 80%는 학습 데이터 prob_3_train과 20%는 테스트 데이터 prob_3_test로  나누고, 표준스케일링 방법으로 학습 데이터의 입력 속성들 중에서 **수치형** 속성을 정규화하고, 학습 데이터에서 사용했던 표준스케일링 설정으로 테스트 데이터의 수치형 입력 속성들을 정규화한다. 

- sklearn.model_selection.train_test_split 함수 이용, random_state=123로 설정한다.

- sklearn.preprocessing.StandardScaler

In [81]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [82]:
X_cols_cont = ['measurement_{}'.format(i) for i in range(0, 18)] + ['loading']
X_cols_bool = ['na_1', 'na_2']
X_cols = X_cols_cont + X_cols_bool
target = 'failure'

df_prob_3_train, df_prob_3_test = train_test_split(df_prob_1, train_size=0.8, random_state=123)
df_prob_3_train = df_prob_3_train.copy()
df_prob_3_test = df_prob_3_test.copy()
df_prob_3_train.shape, df_prob_3_test.shape

((21256, 28), (5314, 28))

In [83]:
std_scaler = StandardScaler()

df_prob_3_train[X_cols_cont] = std_scaler.fit_transform(df_prob_3_train[X_cols_cont])
df_prob_3_test[X_cols_cont] = std_scaler.transform(df_prob_3_test[X_cols_cont])

## 단계 2



결측치 처리 방법를 모두 0 (표준 정규화를 했으므로 평균값이다) 으로 대체한 것과, KNNImputer를 사용한 것과의 성능차이를 비교하여 성능이 좋은 쪽으로 결측치를 처리한 다음에 진행하고자 한다. 모든 속성을 넣었을 때 로지스틱회귀모델은 제대로 된 결과를 내놓지 못하여, 이 보다 유연하게 구분 영역을 만들 수 있는 Decision Tree Classifier를 사용한다. 입력은 measurement_0 ~ measuremenet_17, loading, na_1, na_2 순으로 한다. 
최종적으로 좋은 성능을 보인 결측치 처리 방법으로 prob_3_train, prob_3_test를 교정한다. 문제 4에서도 이 데이터 셋을 사용하여 진행할 예정이다.

- sklearn.tree.DecisionTreeClassifier random_state=123, max_depth=5 로 한다.

- sklearn.impute.KNNImputer

- df_prob_3_train, df_prob_3_test는 성능 비교 후에 처리 될 수 있도록 유의하시오

In [84]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import KNNImputer

In [85]:
df_prob_3_train_simple = df_prob_3_train.fillna(0)
df_prob_3_test_simple = df_prob_3_test.fillna(0)

In [86]:
dct = DecisionTreeClassifier(max_depth=5, random_state=123)
dct.fit(df_prob_3_train_simple[X_cols], df_prob_3_train_simple['failure'])
roc_auc_score(df_prob_3_train_simple['failure'], 
              dct.predict_proba(df_prob_3_train_simple[X_cols])[:, 1])

0.6041446464909006

In [87]:
roc_auc_score(df_prob_3_test_simple['failure'], 
              dct.predict_proba(df_prob_3_test_simple[X_cols])[:, 1])

0.5756305422120515

In [88]:
df_prob_3_train_kni = df_prob_3_train.copy()
df_prob_3_test_kni = df_prob_3_test.copy()
df_prob_3_train_kni.isna().sum().sum()

14676

In [89]:
knn_imp = KNNImputer()

df_prob_3_train_kni[X_cols_cont] = knn_imp.fit_transform(df_prob_3_train_kni[X_cols_cont])
df_prob_3_test_kni[X_cols_cont] = knn_imp.transform(df_prob_3_test_kni[X_cols_cont])

In [90]:
dct = DecisionTreeClassifier(max_depth=5, random_state=123)
dct.fit(df_prob_3_train_kni[X_cols], df_prob_3_train_kni['failure'])
roc_auc_score(df_prob_3_test_kni['failure'], 
              dct.predict_proba(df_prob_3_test_kni[X_cols])[:, 1])

0.5789652194855709

In [91]:
df_prob_3_train = df_prob_3_train_kni
df_prob_3_test = df_prob_3_test_kni

## 단계 3



로지스틱회귀모델을 사용하여 입력 변수 중에서, loading과 measurement_17 만을 사용하여 학습하고 테스트에 대한 성능을 측정하라. 하고 해당 값을 best_auc 변
수에 저장한다.


- sklearn.linear_model.LogisticRegression 모든 설정은 기본


In [92]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [93]:
clf = LogisticRegression()
clf.fit(df_prob_3_train[['loading', 'measurement_17']], df_prob_3_train[target])
roc_auc_score(df_prob_3_train[target], clf.predict_proba(df_prob_3_train[['loading', 'measurement_17']])[:, 1])

0.5901563891548728

In [94]:
best_auc = roc_auc_score(df_prob_3_test[target], clf.predict_proba(df_prob_3_test[['loading', 'measurement_17']])[:, 1])
best_auc

0.5896601269797431

## 단계 4

문제 3 서두에서 설명한 속성 선택법으로 속성을 선택하고 best_auc를 갱신하고 이 값을 소수점 넷째 자리에서 반올림하여, 3째 짜리까지 출력하라. 그리고, 테스트 결과에 대한 failure가 1일 확률을 저장하여 prob_3_test에 저장한다.

Ex) 0.123

In [95]:
sel_feat = ['loading', 'measurement_17']
clf = LogisticRegression()

while True:
    step_best = 0
    step_sel_feat = None
    for i in X_cols:
        if i in sel_feat:
            continue
        cur_feat = sel_feat + [i]
        clf.fit(df_prob_3_train[cur_feat], df_prob_3_train[target])
        cur_auc = roc_auc_score(df_prob_3_test[target], clf.predict_proba(df_prob_3_test[cur_feat])[:, 1])
        if step_best < cur_auc:
            step_best = cur_auc
            step_sel_feat = cur_feat
    if step_best <= best_auc:
        break
    sel_feat = step_sel_feat
    best_auc = step_best
    print(step_best, step_sel_feat)

0.5907461703869561 ['loading', 'measurement_17', 'measurement_1']
0.5917933751557656 ['loading', 'measurement_17', 'measurement_1', 'na_2']
0.592515075145573 ['loading', 'measurement_17', 'measurement_1', 'na_2', 'measurement_2']
0.5932803401689337 ['loading', 'measurement_17', 'measurement_1', 'na_2', 'measurement_2', 'measurement_0']
0.5938082579576055 ['loading', 'measurement_17', 'measurement_1', 'na_2', 'measurement_2', 'measurement_0', 'measurement_9']
0.5941756838066304 ['loading', 'measurement_17', 'measurement_1', 'na_2', 'measurement_2', 'measurement_0', 'measurement_9', 'measurement_4']
0.5943904265427775 ['loading', 'measurement_17', 'measurement_1', 'na_2', 'measurement_2', 'measurement_0', 'measurement_9', 'measurement_4', 'measurement_14']
0.5944999556129847 ['loading', 'measurement_17', 'measurement_1', 'na_2', 'measurement_2', 'measurement_0', 'measurement_9', 'measurement_4', 'measurement_14', 'measurement_7']
0.5945891406345042 ['loading', 'measurement_17', 'measurem

In [96]:
round(best_auc, 3)

0.595

## 단계 5 (Optional)

전처리 단계 3에서 나온 df_optional 데이터프레임으로 KNNImputer를 사용하여 단계 1과 2에서 KNNImputer로 했을 경우와 성능을 비교해보자. 좋은 성능이 나왔다면
속성 선택 과정까지 진행해보자.

In [97]:
X_cols_cont = ['measurement_{}'.format(i) for i in range(0, 18)] + ['loading']
X_cols_bool = ['na_1', 'na_2']
X_cols = X_cols_cont + X_cols_bool
target = 'failure'

df_optional['na_1'] = df_optional['measurement_{}'.format(sel_na_feat[0])].isna() * 1
df_optional['na_2'] = df_optional['measurement_{}'.format(sel_na_feat[1])].isna() * 1

df_optional_train, df_optional_test = train_test_split(df_optional, train_size=0.8, random_state=123)
df_optional_train = df_optional_train.copy()
df_optional_test = df_optional_test.copy()
df_optional_train.shape, df_optional_test.shape

((21256, 28), (5314, 28))

In [98]:
std_scaler = StandardScaler()

df_optional_train[X_cols_cont] = std_scaler.fit_transform(df_optional_train[X_cols_cont])
df_optional_test[X_cols_cont] = std_scaler.transform(df_optional_test[X_cols_cont])

In [99]:
knn_imp = KNNImputer()

df_optional_train[X_cols_cont] = knn_imp.fit_transform(df_optional_train[X_cols_cont])
df_optional_test[X_cols_cont] = knn_imp.transform(df_optional_test[X_cols_cont])

In [100]:
dct = DecisionTreeClassifier(max_depth=5, random_state=123)
dct.fit(df_optional_train[X_cols], df_optional_train['failure'])
roc_auc_score(df_optional_test['failure'], 
              dct.predict_proba(df_optional_test[X_cols])[:, 1])

0.5839618411405162

In [101]:
clf = LogisticRegression()
clf.fit(df_optional_train[['loading', 'measurement_17']], df_optional_train[target])
roc_auc_score(df_optional_train[target], clf.predict_proba(df_optional_train[['loading', 'measurement_17']])[:, 1])

best_auc = roc_auc_score(df_optional_test[target], clf.predict_proba(df_optional_test[['loading', 'measurement_17']])[:, 1])
best_auc

0.5899887141903644

In [102]:
sel_feat = ['loading', 'measurement_17']
clf = LogisticRegression()

while True:
    step_best = 0
    step_sel_feat = None
    for i in X_cols:
        if i in sel_feat:
            continue
        cur_feat = sel_feat + [i]
        clf.fit(df_optional_train[cur_feat], df_optional_train[target])
        cur_auc = roc_auc_score(df_optional_test[target], clf.predict_proba(df_optional_test[cur_feat])[:, 1])
        if step_best < cur_auc:
            step_best = cur_auc
            step_sel_feat = cur_feat
    if step_best <= best_auc:
        break
    sel_feat = step_sel_feat
    best_auc = step_best
    print(step_best, step_sel_feat)

0.5910702366978691 ['loading', 'measurement_17', 'measurement_1']
0.5918985888217057 ['loading', 'measurement_17', 'measurement_1', 'measurement_2']
0.5926476197052044 ['loading', 'measurement_17', 'measurement_1', 'measurement_2', 'measurement_0']
0.5932495158527403 ['loading', 'measurement_17', 'measurement_1', 'measurement_2', 'measurement_0', 'measurement_9']
0.5935676227958558 ['loading', 'measurement_17', 'measurement_1', 'measurement_2', 'measurement_0', 'measurement_9', 'measurement_4']
0.5938425756963007 ['loading', 'measurement_17', 'measurement_1', 'measurement_2', 'measurement_0', 'measurement_9', 'measurement_4', 'measurement_14']
0.5940320424931693 ['loading', 'measurement_17', 'measurement_1', 'measurement_2', 'measurement_0', 'measurement_9', 'measurement_4', 'measurement_14', 'na_2']
0.5941471199402912 ['loading', 'measurement_17', 'measurement_1', 'measurement_2', 'measurement_0', 'measurement_9', 'measurement_4', 'measurement_14', 'na_2', 'measurement_11']
0.59424103

# 문제 4



PCA를 사용하여 차원 감소로 어떤 효과를 볼 수 있을 지 검증하고자 한다.

## 단계 1

진행하기 전에 loading을 포함하지 포함하지 않을지를 확인하고자 한다. 분석가들 사이에 loading은 흡수력하고는 상관이 없으니 빼고 해야 하는 쪽과 loading에 따라 흡수력 세팅을 달리 했을 테니 포함하는 것이 좋다는 쪽으로 나뉘 었다. 이를 위해 흡수력에 따른 loading과의 상관도를 파악해 보기 위해 LinearDiscriminantAnalysis를 통해 loading을 제외하고 failure를 나누었을 때 나온 투영점과 loading과의 spearman correlation을 구하고, 이 때 pvalue를 F에 저장한다. prob_3_train 데이터프레임을 사용한다.

- sklearn.discriminant_analysis.LinearDiscriminatAnalysis
- scipy.stats.spearmanr

In [103]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy.stats import spearmanr

In [104]:
lda = LinearDiscriminantAnalysis()
all_m_cols = ['measurement_{}'.format(i) for i in range(0, 18)]

In [105]:
lda.fit(df_prob_3_train[all_m_cols], df_prob_3_train['failure'])

LinearDiscriminantAnalysis()

In [106]:
roc_auc_score(df_prob_3_train['failure'], lda.predict_proba(df_prob_3_train[all_m_cols])[:, 1])

0.5335202939418473

In [107]:
roc_auc_score(df_prob_3_test['failure'], lda.predict_proba(df_prob_3_test[all_m_cols])[:, 1])

0.5216738096882059

In [108]:
res = spearmanr(lda.transform(df_prob_3_train[all_m_cols])[:, 0],
          df_prob_3_train['loading'])
F = res.pvalue
res, F

(SpearmanrResult(correlation=0.0045465663583843, pvalue=0.507440560217586),
 0.507440560217586)

In [109]:
from scipy.stats import pearsonr

In [110]:
pearsonr(lda.transform(df_prob_3_train[all_m_cols])[:, 0],
          df_prob_3_train['loading'])

(0.004114160360302859, 0.5486470618570178)

## 단계 2

prob_3_train으로 measurement_0~measurement_17까지 PCA를 진행하고, 50%이상의 분산을 설명할 수 있는 pca의 컴포넌트의 수를 G에 저장한다.

- sklearn.decomposition.PCA

In [111]:
from sklearn.decomposition import PCA

In [112]:
pca_cols = ['measurement_{}'.format(i) for i in range(0, 18)]
pca = PCA()

In [113]:
pca.fit(df_prob_3_train[pca_cols])
c_var_ratio_ = np.cumsum(pca.explained_variance_ratio_)
c_var_ratio_

array([0.10486835, 0.18159522, 0.24908721, 0.31218862, 0.37189656,
       0.42910798, 0.48563414, 0.54171955, 0.59776626, 0.65293746,
       0.70778585, 0.76247117, 0.81405106, 0.86378294, 0.90768346,
       0.95061148, 0.99097572, 1.        ])

In [114]:
G = np.sum(c_var_ratio_ < 0.5) + 1
c_var_ratio_[G - 1], G

(0.5417195525921475, 8)

## 단계 3

로지스틱회귀 분석을 사용하여 'loading' 속성과 PCA 컴포넌트수를 G부터 증가시켜서 failure에 대한 roc_auc_score에 대한 가장 좋은 prob_4_test에 대한 성능을 보이는 컴포넌트수 H를 구하라

In [115]:
pca_train = pca.transform(df_prob_3_train[pca_cols])
pca_test = pca.transform(df_prob_3_test[pca_cols])

In [116]:
df_pca_train = pd.DataFrame(pca_train,
                            index=df_prob_3_train.index,
                            columns=['pca_{}'.format(i) for i in range(0, 18)])
df_pca_test = pd.DataFrame(pca_test,
                            index=df_prob_3_test.index,
                            columns=['pca_{}'.format(i) for i in range(0, 18)])

df_pca_train[['loading', 'failure']] = df_prob_3_train[['loading', 'failure']]
df_pca_test[['loading', 'failure']] = df_prob_3_test[['loading', 'failure']]

In [117]:
best_score = 0
best_comp = 0
lr = LogisticRegression()
for i in range(G, 18):
    sel_pca = df_pca_train.columns[:i].tolist() + ['loading']
    lr.fit(df_pca_train[sel_pca], df_pca_train['failure'])
    score_ = roc_auc_score(df_pca_test['failure'], lr.predict_proba(df_pca_test[sel_pca])[:, 1])
    if score_ > best_score:
        best_score = score_
        best_comp = i
    print(i, score_, best_score)
H = best_comp

8 0.5946281847683491 0.5946281847683491
9 0.5955888759563758 0.5955888759563758
10 0.5949347839667525 0.5955888759563758
11 0.5947948415712346 0.5955888759563758
12 0.5944444718438366 0.5955888759563758
13 0.5940910196848194 0.5955888759563758
14 0.5940558799643588 0.5955888759563758
15 0.5940392348336145 0.5955888759563758
16 0.5941493803901454 0.5955888759563758
17 0.5941304748095467 0.5955888759563758


F + G + H 를 소수점 셋째 자리에서 반올림하여, 2째 자리까지 구하라.

In [118]:
round(F + G + H, 2)

17.51

# 문제 5

[Kaggle 형] train.csv을 학습 데이터로 사용하여, failure를 맞추는 분류 모델을 만들고, test.csv 테스트 데이터에 대한 failure일 확률을 다음과 같은 포맷으로 submission.csv라는 이름으로 만들어라. 측정에 대한 지표는 area under of ROC curve 이다.

Id,failure

26570, 0.1

26571, 0.2

....




In [207]:
df_prob_5_train = df_org.copy()
df_prob_5_test = df_test.copy()

In [209]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer

In [210]:
def preproc(df, std_scaler=None, knn_imputer=None):
    # 전처리 단계의 코드를 가져와서 df_test에 대해서도 처리한다.
    lr = Lasso()
    pc_list = [] # coef 데이터프레임 구성을 위해 처리한 product_code 저장 
    coef_list = [] # 마찬가지로 회귀계수 저장
    impute_cnt = []
    for k, v in df.dropna().groupby('product_code'): # groupby를 이용하여 product_code 별로 구분하여 데이터프레임의 행들을 나누어 가져온다.
        lr.fit(v[m_cols],v['measurement_17']) # 학습
        print(k, lr.coef_, r2_score(v['measurement_17'], lr.predict(v[m_cols]))) # 결과 확인
        coef_ = lr.coef_.copy() # 복사
        avail_cols = np.array(m_cols)[coef_ != 0] # 연관성이 있는 속성명을 뽑아 낸다, 작업을 편하게 하기 위해 list를 numpy array로 바꾼다.
        pc_list.append(k) # 처리한 product_code를 저장
        coef_list.append(coef_) # 도출한 회귀계수 저장 
        # 현재 복원 스텝을 진행하는 product_code 에서 연관성이 있는 속성이 결측이 없고 measurement_17은 결측인 행을 뽑기 위한  Boolean 인덱싱 행을  만든다.
        target_cond = (df['product_code'] == k) & df[avail_cols].notna().all(axis=1) & df['measurement_17'].isna()
        impute_cnt.append(target_cond.sum())
        # 결측치를 예측하고 입력한다. loc를 행을 선택후 값을 지정할 때에, 대입하는 값의 수가 일치하면, 인덱스와 상관없이 순서대로 대입이 되게 된다. 
        df.loc[target_cond, 'measurement_17'] = lr.predict(df.loc[target_cond][m_cols].fillna(0))
    df_coef = pd.DataFrame(np.vstack(coef_list), columns=lr.feature_names_in_, index=pc_list) # coef 데이터프레밍 생성
    df[['na_1', 'na_2']] = df[['measurement_3', 'measurement_5']].isna() * 1
    X_std_cols = ['loading'] + ['measurement_{}'.format(i) for i in range(18)]
    if std_scaler is None:
        std_scaler = StandardScaler()
        knn_imputer = KNNImputer()
        df[X_std_cols] = std_scaler.fit_transform(df[X_std_cols])
        df[X_std_cols] = knn_imputer.fit_transform(df[X_std_cols])
    else:
        df[X_std_cols] = std_scaler.transform(df[X_std_cols])
        df[X_std_cols] = knn_imputer.transform(df[X_std_cols])
    return df, std_scaler, knn_imputer, df_coef

In [211]:
df_prob_5_train_proc, std_scaler, knn_imputer, df_coef = preproc(df_prob_5_train)
df_prob_5_test_proc, _, _, _ = preproc(df_prob_5_test, std_scaler, knn_imputer)

A [-0.         13.83187198 58.81809297 25.74675741 24.09935052 76.3016922
  0.        ] 0.9995187098133933
B [17.70043689 43.4580317  33.95620071  0.         86.48501665  0.
 25.75857943] 0.9995778604871972
C [ 0.          0.         51.38813886  6.16278066 40.71868984 90.19970609
 35.5609767 ] 0.9996273421906502
D [23.74687005 -0.         62.67768903 92.76582051 47.32698957 56.1575785
  0.        ] 0.9997306028113272
E [ 0.         57.70894625 65.67813663 76.08679497 -0.         72.16934824
 18.93095817] 0.9997401972700377
F [-0.         90.16468094 52.85990792 95.44555137 83.58526862  0.
 -0.        ] 0.9998193750765353
G [-0.         63.23214378 18.89469775 78.03443654 -0.         64.65918236
 49.00161611] 0.9997141225901649
H [ 0.         57.99157424 58.42094608  0.         34.01989317 36.73290873
 59.56024093] 0.9996183553658226
I [34.8576455  18.57903816 -0.         -0.         32.25403768 73.67412295
 19.35946393] 0.999400268363214


In [215]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(df_prob_5_train_proc[sel_feat], df_prob_5_train_proc['failure'])
roc_auc_score(df_prob_5_train_proc['failure'], clf.predict_proba(df_prob_5_train_proc[sel_feat])[:, 1])

0.5924246088444954

In [223]:
pd.DataFrame({'failure': clf.predict_proba(df_prob_5_test_proc[sel_feat])[:, 1]}, index=df_prob_5_test_proc.index).to_csv('submission.csv', index=True)
!head submission.csv

id,failure
26570,0.2024252783178391
26571,0.15894565080828824
26572,0.18712591195503184
26573,0.18905675387948673
26574,0.33866672114329643
26575,0.16340052766725438
26576,0.15571856119140876
26577,0.23702050971513688
26578,0.13477324581162625


In [224]:
!kaggle competitions submit -c tabular-playground-series-aug-2022 -f submission.csv -m "Message"

100% 520k/520k [00:00<00:00, 2.05MB/s]
Successfully submitted to Tabular Playground Series - Aug 2022

# 문제 6 [Optional]


학습 / 검증 분리를 통한 결과 보다 치우침이 없는 교차 검증을 사용하여 최적화된 모델을 탐색하고자 한다. 다음과 같은 과정으로 진행한다.

## 단계 1
prob_1 데이터셋을 prob_6 데이터셋으로 복사하고, 문제 3의 2단계에서 수행했던, KNNImputer를 통한 결측치 보간을 진행한다.

- sklearn.imputer.KNNImputer

In [151]:
df_prob_6 = df_prob_1.copy()
imputer = KNNImputer()
X_cols_impute = ['measurement_{}'.format(i) for i in range(3, 18)] + ['loading']
df_prob_6[X_cols_impute] = imputer.fit_transform(df_prob_6[X_cols_impute])

## 단계 2

loading과, measurement_0~measurement_17은 표준정규화를 수행하고, 학습에  대한 AUC와  테스트에 대한 failure가 1일 확률을 예측한 결과를 넘겨주는 함수를 만들어라.

- sklearn.preprocessing.StandardScaler
- sklearn.metrics.roc_auc_score

In [152]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [173]:
def train_n_eval(clf, df_train, df_test):
    X_std_cols = ['measurement_{}'.format(i) for i in range(18)] + ['loading']
    X_std_cols = [i for i in X_std_cols if i in df_train.columns]
    df_train_proc = df_train.copy() # 넘어온 train 데이터를 유지하기 위함
    df_test_proc = df_test.copy()
    std_scaler = StandardScaler()
    df_train_proc[X_std_cols] = std_scaler.fit_transform(df_train_proc[X_std_cols]) # 표준 스케일링
    df_test_proc[X_std_cols] = std_scaler.transform(df_test_proc[X_std_cols])
    X_cols =[i for i in df_train.columns if i != 'failure']
    clf.fit(df_train_proc[X_cols], df_train_proc['failure']) # 학습
    y_test_hat = clf.predict_proba(df_test_proc[X_cols])[:, 1]
    return roc_auc_score(df_train_proc['failure'], clf.predict_proba(df_train_proc[X_cols])[:, 1]), \
        y_test_hat, roc_auc_score(df_test_proc['failure'], y_test_hat)

## 단계 3

product_code에 대한 그룹 5-fold 교차검증을 2단계에서 작성한 함수를 사용하여 LogisticRegression으로 진행한다.

- sklearn.model_selection.GroupKFold, random_state=123
- loading, measurement_0~measurement_17, na_1, na_2를 입력으로 하라.

In [174]:
from sklearn.model_selection import GroupKFold

In [181]:
kf = GroupKFold(n_splits=5) # GroupKFold 생성
X_cols = ['loading'] + ['measurement_{}'.format(i) for i in range(18)] + ['na_1', 'na_2']
train_auc_list = []
auc_list = []
y_hat_list = []
for train_index, test_index in kf.split(df_prob_6[X_cols], df_prob_6['failure'], df_prob_6['product_code']): # product에 대한 GroupKFold를 수행
    train_auc, y_hat, auc = train_n_eval(LogisticRegression(), 
                              df_prob_6.loc[train_index, X_cols + ['failure']], 
                              df_prob_6.loc[test_index, X_cols + ['failure']])
    train_auc_list.append(train_auc)
    y_hat_list.append(y_hat)
    auc_list.append(auc)
np.mean(auc_list)

0.5875354299791633

## 단계 4

최적의 RandomForestClassifier의 Hyper Parameter Tuning을 진행하고자 한다. 

탐색할 매개 변수는, 

n_estimators=[5, 10, 15]와, Decision Tree의 max_depth=[5, 6, 7], 그리고 min_samples_split = [512, 1024] 로 한다.

* sklearn.ensemble.RandomForesetClassifier, random_state=123
* itertools의 product를 이용
* loading, measurement_0~measurement_17, na_1, na_2를 입력으로 순서에 맞춰 진행 하라.

교차 검증을 통해 도출된 최적의 auc에 해당하는 
n_estimators + max_dpeth + min_samples_split + 최적 AUC를 소수점 셋째 자리에서 반올림하여 둘째 자리까지 출력하라.

In [183]:
import itertools
from sklearn.ensemble import RandomForestClassifier

In [187]:
best_auc = 0
best_param = None
for n_estimators, max_depth, min_samples_split in itertools.product([5, 10, 15], [5, 6, 7], [512,1024]):
    clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, min_samples_split=min_samples_split)
    train_auc_list = []
    auc_list = []
    for train_index, test_index in kf.split(df_prob_6[X_cols], df_prob_6['failure'], df_prob_6['product_code']):
        train_auc, y_hat, auc = train_n_eval(clf, 
                                df_prob_6.loc[train_index, X_cols + ['failure']], 
                                df_prob_6.loc[test_index, X_cols + ['failure']])
        train_auc_list.append(train_auc)
        auc_list.append(auc)
    cur_auc = np.mean(auc_list)
    if best_auc < cur_auc:
        best_auc = cur_auc
        best_param = (n_estimators, max_depth, min_samples_split)
    print(n_estimators, max_depth, min_samples_split, np.mean(train_auc_list), cur_auc)

5 5 512 0.6055393223318555 0.5737344879912246
5 5 1024 0.6039404926463068 0.5777198812341875
5 6 512 0.6087399969392931 0.5706951119453634
5 6 1024 0.608772945092771 0.5730186130334397
5 7 512 0.6202192144761458 0.5653496608739041
5 7 1024 0.6101634341816926 0.5694603650855581
10 5 512 0.6123923004633928 0.576255440956406
10 5 1024 0.6083436799805046 0.5715198121626515
10 6 512 0.6251752690506209 0.5763895858256882
10 6 1024 0.6142630768745725 0.5736676864383385
10 7 512 0.6288665013494796 0.5748929692238216
10 7 1024 0.6206602803316887 0.5824068663317374
15 5 512 0.6181442508984242 0.5761195983680165
15 5 1024 0.6155122936024915 0.5753331265486066
15 6 512 0.6274991824829566 0.5763782103897451
15 6 1024 0.6206188728709401 0.5806779509805727
15 7 512 0.6344697433358247 0.5776206200697062
15 7 1024 0.6257432627446502 0.5782384380676955


In [188]:
round(np.sum(best_param) + best_auc, 2)

1041.58